In [ ]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score, classification_report
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim

# Disable GPU and force TensorFlow to run on CPU
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'


# Load the data
train_data = np.load('../Files/extracted_features/layer_features_lv60_self/train_0.npz')
test_data = np.load('../Files/extracted_features/layer_features_lv60_self/test_0.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = balanced_accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))


# # SVM Classification
# from sklearn.svm import SVC

# svm_model = SVC(kernel='linear', C=10.0, gamma='scale', random_state=42)
# svm_model.fit(train_features, train_labels_encoded)
# svm_predictions = svm_model.predict(test_features)

# svm_accuracy = accuracy_score(test_labels_encoded, svm_predictions)
# print(f"SVM Accuracy: {svm_accuracy:.4f}")
# print("SVM Classification Report:")
# print(classification_report(test_labels_encoded, svm_predictions))

# ANN Model
# ann_model = Sequential([
#     Dense(512, activation='relu', input_shape=(train_features_normalized.shape[1],)),
#     Dropout(0.5),
#     Dense(256, activation='relu'),
#     Dropout(0.5),
#     Dense(128, activation='relu'),
#     Dropout(0.5),
#     Dense(len(label_encoder.classes_), activation='softmax')
# ])

# ann_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# # Train the ANN
# ann_history = ann_model.fit(
#     train_features_normalized, 
#     train_labels_encoded, 
#     epochs=50, 
#     batch_size=32, 
#     validation_split=0.2,
#     callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
# )

# # Evaluate the ANN
# ann_test_loss, ann_test_accuracy = ann_model.evaluate(test_features_normalized, test_labels_encoded)
# print(f"ANN Test Accuracy: {ann_test_accuracy:.4f}")

# # ANN Predictions and Classification Report
# ann_predictions = np.argmax(ann_model.predict(test_features_normalized), axis=1)
# print("ANN Classification Report:")
# print(classification_report(test_labels_encoded, ann_predictions))

# PyTorch CNN Model
# class CNN1DModel(nn.Module):
#     def __init__(self, input_channels, num_classes):
#         super(CNN1DModel, self).__init__()
#         self.conv1 = nn.Conv1d(input_channels, 32, kernel_size=5, stride=1, padding=2)
#         self.bn1 = nn.BatchNorm1d(32)
#         self.relu = nn.ReLU()
#         self.conv2 = nn.Conv1d(32, 64, kernel_size=5, stride=1, padding=2)
#         self.bn2 = nn.BatchNorm1d(64)
#         self.conv3 = nn.Conv1d(64, 128, kernel_size=5, stride=1, padding=2)
#         self.bn3 = nn.BatchNorm1d(128)
#         self.flatten = nn.Flatten()
#         self.fc1 = nn.Linear(128 * train_features_cnn.shape[1], 512)  # Adjusted based on your data
#         self.fc2 = nn.Linear(512, num_classes)
#         self.dropout = nn.Dropout(0.5)
    
#     def forward(self, x):
#         x = self.relu(self.bn1(self.conv1(x)))
#         x = self.relu(self.bn2(self.conv2(x)))
#         x = self.relu(self.bn3(self.conv3(x)))
#         x = self.flatten(x)
#         x = self.dropout(self.relu(self.fc1(x)))
#         x = self.fc2(x)
#         return x

# # Prepare tensors for PyTorch
# X_train_tensor = torch.tensor(train_features_cnn, dtype=torch.float32).permute(0, 2, 1)  # Channels first for PyTorch
# y_train_tensor = torch.tensor(train_labels_encoded, dtype=torch.long)
# X_test_tensor = torch.tensor(test_features_cnn, dtype=torch.float32).permute(0, 2, 1)
# y_test_tensor = torch.tensor(test_labels_encoded, dtype=torch.long)

# train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
# test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

# train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
# test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# # Initialize PyTorch model, loss, and optimizer
# input_channels = 1
# num_classes = len(label_encoder.classes_)
# model = CNN1DModel(input_channels, num_classes)
# criterion = nn.CrossEntropyLoss()
# optimizer = optim.Adam(model.parameters(), lr=0.001)

# # PyTorch Training Loop
# epochs = 100
# for epoch in range(epochs):
#     model.train()
#     running_loss = 0.0
#     for inputs, labels in train_loader:
#         optimizer.zero_grad()
#         outputs = model(inputs)
#         loss = criterion(outputs, labels)
#         loss.backward()
#         optimizer.step()
#         running_loss += loss.item() * inputs.size(0)

#     epoch_loss = running_loss / len(train_loader.dataset)
#     print(f"Epoch {epoch+1}: Loss = {epoch_loss:.4f}")

# # Test PyTorch model
# def test_model(model, test_loader, criterion):
#     model.eval()
#     test_loss = 0.0
#     correct = 0
#     total = 0

#     with torch.no_grad():
#         for inputs, labels in test_loader:
#             outputs = model(inputs)
#             loss = criterion(outputs, labels)
#             test_loss += loss.item() * inputs.size(0)
#             _, predicted = torch.max(outputs, 1)
#             total += labels.size(0)
#             correct += (predicted == labels).sum().item()

#     avg_loss = test_loss / total
#     accuracy = correct / total
#     return avg_loss, accuracy

# # Evaluate PyTorch model
# test_loss, test_accuracy = test_model(model, test_loader, criterion)
# print(f"PyTorch CNN Test Accuracy: {test_accuracy:.4f}")


In [ ]:
import torch

torch.cuda.is_available()
# True
torch.cuda.device_count()
# 1
torch.cuda.current_device()
# 0
torch.cuda.device(0)
# <torch.cuda.device at 0x7efce0b03be0>
torch.cuda.get_device_name(0)
# 'GeForce GTX 950M'

In [ ]:
# base 960h

In [1]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, balanced_accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('./train_0.npz')
test_data = np.load('./test_0.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../y_train.csv').Gender_bin
test_labels = pd.read_csv('../y_test_age_group.csv').Gender_bin  

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = balanced_accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))


2024-12-28 15:28:47.456557: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-28 15:28:47.469005: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-28 15:28:47.472429: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-28 15:28:47.482741: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-12-28 15:28:48.299568: W tensorflow/compiler/tf2

Epoch 1/50


/home/mojo/.local/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-12-28 15:28:50.372547: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:266] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 362ms/step - accuracy: 0.6031 - loss: 1.4178 - val_accuracy: 0.7907 - val_loss: 0.5343
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 369ms/step - accuracy: 0.8442 - loss: 0.3790 - val_accuracy: 0.7907 - val_loss: 0.6223
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 362ms/step - accuracy: 0.9351 - loss: 0.1760 - val_accuracy: 0.7849 - val_loss: 0.7865
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 358ms/step - accuracy: 0.9614 - loss: 0.1115 - val_accuracy: 0.7907 - val_loss: 0.8392
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 363ms/step - accuracy: 0.9810 - loss: 0.0590 - val_accuracy: 0.7965 - val_loss: 0.7601
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 354ms/step - accuracy: 0.9939 - loss: 0.0323 - val_accuracy: 0.7965 - val_loss: 1.0268
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
TensorFlow CNN Balanced Accuracy Score: 0.8711
TensorFlow CNN Classification Report:
              precision    recall  f1-score   support

           0       0.82      0.89      0.85        56
           

In [2]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, balanced_accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('../Files/extracted_features/layer_features_base_100/train_1.npz')
test_data = np.load('../Files/extracted_features/layer_features_base_100/test_1.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = balanced_accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))


Epoch 1/50


/home/mojo/.local/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 365ms/step - accuracy: 0.6637 - loss: 1.5025 - val_accuracy: 0.8081 - val_loss: 0.3940
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 357ms/step - accuracy: 0.9002 - loss: 0.2663 - val_accuracy: 0.7907 - val_loss: 0.5852
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 359ms/step - accuracy: 0.9703 - loss: 0.1058 - val_accuracy: 0.8372 - val_loss: 0.6529
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 358ms/step - accuracy: 0.9901 - loss: 0.0479 - val_accuracy: 0.8256 - val_loss: 0.7855
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 361ms/step - accuracy: 0.9915 - loss: 0.0262 - val_accuracy: 0.8140 - val_loss: 0.7427
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 359ms/step - accuracy: 0.9989 - loss: 0.0144 - val_accuracy: 0.8081 - val_loss: 0.9622
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
TensorFlow CNN Balanced Accuracy Score: 0.8505
TensorFlow CNN Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.89      0.83        56
           

In [3]:
import os
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, balanced_accuracy_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Flatten, Dense, Dropout

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

# Load the data
train_data = np.load('../Files/extracted_features/layer_features_base_100/train_2.npz')
test_data = np.load('../Files/extracted_features/layer_features_base_100/test_2.npz')

train_features = train_data['features']
test_features = test_data['features']

train_labels = pd.read_csv('../Files/labels/y_train.csv').Gender_bin
test_labels = pd.read_csv('../Files/labels/y_test.csv').Gender_bin

# Encode the labels
label_encoder = LabelEncoder()
train_labels_encoded = label_encoder.fit_transform(train_labels)
test_labels_encoded = label_encoder.transform(test_labels)

# Normalize the features
scaler = StandardScaler()
train_features_normalized = scaler.fit_transform(train_features)
test_features_normalized = scaler.transform(test_features)

# Reshape the data for CNN
train_features_cnn = np.expand_dims(train_features_normalized, axis=-1)
test_features_cnn = np.expand_dims(test_features_normalized, axis=-1)

# TensorFlow CNN Model
cnn_model = Sequential([
    Conv1D(64, kernel_size=5, activation='relu', input_shape=(train_features_cnn.shape[1], 1)),
    Conv1D(128, kernel_size=5, activation='relu'),
    Conv1D(256, kernel_size=5, activation='relu'),
    Flatten(),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(len(label_encoder.classes_), activation='softmax')
])

cnn_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the TensorFlow CNN
history = cnn_model.fit(
    train_features_cnn, 
    train_labels_encoded, 
    epochs=50, 
    batch_size=32, 
    validation_split=0.2,
    callbacks=[tf.keras.callbacks.EarlyStopping(patience=5)]
)

# Evaluate the TensorFlow CNN
cnn_predictions = np.argmax(cnn_model.predict(test_features_cnn), axis=1)
cnn_balanced_accuracy = balanced_accuracy_score(test_labels_encoded, cnn_predictions)

print(f"TensorFlow CNN Balanced Accuracy Score: {cnn_balanced_accuracy:.4f}")
print("TensorFlow CNN Classification Report:")
print(classification_report(test_labels_encoded, cnn_predictions))


Epoch 1/50


/home/mojo/.local/lib/python3.12/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


22/22 ━━━━━━━━━━━━━━━━━━━━ 9s 370ms/step - accuracy: 0.6397 - loss: 1.2704 - val_accuracy: 0.8256 - val_loss: 0.3737
Epoch 2/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 363ms/step - accuracy: 0.9528 - loss: 0.2045 - val_accuracy: 0.9186 - val_loss: 0.1882
Epoch 3/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 362ms/step - accuracy: 0.9620 - loss: 0.1087 - val_accuracy: 0.9012 - val_loss: 0.3353
Epoch 4/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 360ms/step - accuracy: 0.9932 - loss: 0.0164 - val_accuracy: 0.8953 - val_loss: 0.4840
Epoch 5/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 361ms/step - accuracy: 0.9938 - loss: 0.0172 - val_accuracy: 0.8314 - val_loss: 0.7282
Epoch 6/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 359ms/step - accuracy: 0.9934 - loss: 0.0168 - val_accuracy: 0.8605 - val_loss: 0.6717
Epoch 7/50
22/22 ━━━━━━━━━━━━━━━━━━━━ 8s 377ms/step - accuracy: 0.9954 - loss: 0.0296 - val_accuracy: 0.8663 - val_loss: 0.3875
3/5 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/stepWARNING:tensorflow:5 out of the last 11 calls to <function TensorFlowTrainer.m